### Purpose
Playing around with NB Multinormial language modeling. Eventually need to turn this into a .py file.

In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [2]:
df = pickle.load(open("../movies.p", 'rb'))

In [3]:
df.shape
df.head()

(304354, 9)

,gender_to,movie_id,gender_from,char_id_from,char_id_to,line_id,words,movie_year,genre
0,m,m0,f,u0,u2,L194,we make quick roxanne korrine andrew barrett i...,1999,comedy
1,m,m0,f,u0,u2,L195,well i think we start pronunciation okay you,1999,comedy
2,m,m0,f,u0,u2,L196,hacking gagging spit part please,1999,comedy
3,m,m0,f,u0,u2,L197,okay bout we try french cuisine saturday night,1999,comedy
4,m,m0,f,u0,u2,L198,you ask me cute your name,1999,comedy


In [4]:
# remove unknown gender. 0 is male and 1 is female.
df = df[df.gender_from != '?']
df['gender_from'] = np.where(df.gender_from == 'f', 1, 0)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df.words, df.gender_from.astype('int'), test_size=0.33, random_state=42)

In [6]:
y_train.value_counts()
y_test.value_counts()

0    125899
1     54277
Name: gender_from, dtype: int64

0    62301
1    26443
Name: gender_from, dtype: int64

In [7]:
X_train.shape
X_test.shape

(180176,)

(88744,)

In [8]:
# transform
count_vect = CountVectorizer() # using bag of words
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [82]:
# train
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [83]:
# test
X_test_counts = count_vect.transform(X_test) # JW TO DO: how does this treat unseen ngrams?
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

predicted = clf.predict(X_test_tfidf)
proba = clf.predict_proba(X_test_tfidf)

In [84]:
np.bincount(predicted)
predicted
proba

array([86541,  2203], dtype=int64)

array([0, 0, 0, ..., 0, 0, 0])

array([[0.897381  , 0.102619  ],
       [0.79208409, 0.20791591],
       [0.739961  , 0.260039  ],
       ...,
       [0.81081729, 0.18918271],
       [0.78085247, 0.21914753],
       [0.57516826, 0.42483174]])

### Results: Classifier doesn't do better than baseline; 70% accuracy

Using unigrams and bigrams does worse than simple bag of words approach

In [85]:
baseline = np.mean(df.gender_from) # percent of female lines in whole dataset

In [86]:
print('baseline (% female lines): ', baseline)
print('accuracy: ', metrics.accuracy_score(y_test, predicted))
print('precision: ', metrics.precision_score(y_test, predicted))
print('recall: ', metrics.recall_score(y_test, predicted))
metrics.confusion_matrix(y_test, predicted)

baseline (% female lines):  0.30016361743269376
accuracy:  0.7097719282430361
precision:  0.6559237403540626
recall:  0.0546458419997731


array([[61543,   758],
       [24998,  1445]], dtype=int64)